# Dadbot: Dad's memorial bot based on RASA (old style)

## Starting Jupyter Notebook with necessary imports

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')


# Installations
* Rasa
* SpaCy Language Model

In [2]:
import sys
python = sys.executable

# In your environment run:
#!{python} -m pip install -U pip
#!{python} -m pip install rasa[convert]~=2.0.0

In [3]:
#!{python} -m spacy download es_core_news_md

## Downloading the Spanish Language Model

In [4]:
#!{python} -m spacy link es_core_news_md es --force;

## Import the Installations

In [5]:
import rasa


# 1. Teaching the bot to understand user inputs using Rasa NLU

## Training the NLU Model.

In [17]:
from rasa.model import get_model
from rasa.shared.nlu.training_data.loading import load_data
from rasa.shared.core.slots import Slot, TextSlot
from rasa.shared.core.domain import Domain
from rasa.nlu import config, utils
from rasa.nlu.components import ComponentBuilder
from rasa.nlu.config import RasaNLUModelConfig
from rasa.nlu.model import Interpreter, Trainer, TrainingData
from rasa.nlu.components import Component
from rasa.nlu.tokenizers.tokenizer import Token
from rasa.utils.tensorflow.constants import ENTITY_RECOGNITION

import spacy

#spacy_parser = spacy.load('es_core_news_md')
#nlp = spacy.load('es')

# loading the nlu training samples
training_data = load_data("data/nlu/nlu.yml")

# trainer to train our pipeline
trainer = Trainer(config.load("config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/", fixed_model_name="current")

INFO:rasa.nlu.model:Starting to train component WhitespaceTokenizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component RegexFeaturizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component LexicalSyntacticFeaturizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component CountVectorsFeaturizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component CountVectorsFeaturizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component DIETClassifier
Epochs: 100%|██████████| 100/100 [01:14<00:00,  1.35it/s, t_loss=1.497, i_acc=0.991, e_f1=0.997]
INFO:rasa.utils.tensorflow.models:Finished training.
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component EntitySynonymMapper
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Startin

## Evaluating the NLU model on a random text

In [ ]:
# A helper function for prettier output

def pprint(o):   
    print(json.dumps(o, indent=2))
    
pprint(interpreter.parse("dejándome el coche"))

## Evaluating the NLU model on a test data
(Here we are using the data at hand i.e nlu.md but it isr recommended to use unseen data)

In [ ]:
from rasa.nlu.test import run_evaluation

run_evaluation("data/nlu/nlu.yml", model_directory)

# 2. Teaching the bot to respond using Rasa Core

##  Visualising the Training Data

In [ ]:
#!apt-get -qq install -y graphviz libgraphviz-dev pkg-config;
#!breq install graphviz

#!conda install -y -n rasa pygraphviz pkg-config;

In [ ]:
from IPython.display import Image
from rasa.core.agent import Agent
 
agent = Agent('domain.yml')
agent.visualize("data/core/stories.yml", "story_graph.png", max_history=2)
Image(filename="story_graph.png")

## Training a Dialogue Model

In [14]:
from rasa.core.policies.policy import Policy
from rasa.core.policies.registry import RulePolicy, MemoizationPolicy, FallbackPolicy, TEDPolicy
from rasa.core.agent import Agent

import asyncio

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
# Deprecated
#fallback = FallbackPolicy(fallback_action_name="utter_unclear",
#                          core_threshold=0.2,
#                          nlu_threshold=0.1)

agent = Agent('domain.yml',
              policies=[MemoizationPolicy(max_history=5),
                        RulePolicy()])
#                        fallback])

# loading our neatly defined training dialogues
training_data = await agent.load_data('data/core/stories.yml')

agent.train(training_data)

agent.persist('./models/')

Processed trackers: 100%|██████████| 29/29 [00:00<00:00, 340.37it/s, # actions=147]
Processed actions: 147it [00:00, 2578.31it/s, # examples=143]
Processed trackers: 0it [00:00, ?it/s]
Processed trackers: 100%|██████████| 29/29 [00:00<00:00, 2212.95it/s]
INFO:rasa.core.agent:Model directory ./models/core exists and contains old model files. All files will be overwritten.
INFO:rasa.core.agent:Persisted model to '/home/debian/workspace/Dadbot/models/core'


# Talk to your Bot

In [15]:
#Starting the Bot
from rasa.core.agent import Agent
from rasa.core.utils import EndpointConfig

action_endpoint = EndpointConfig(url="http://0.0.0.0:5055/webhook")
agent = Agent.load('./models/', interpreter=model_directory, action_endpoint=action_endpoint)

#!docker run -d -p 5055:5055 --mount type=bind,source=/home/debian/workspace/Dadbot/actions,target=/app/actions rasa/rasa-sdk

## Creating the voice synthesizer

In [10]:
#!git clone https://github.com/NVIDIA/tacotron2.git
#!git clone https://github.com/DeepLearningExamples/CUDA-Optimized/FastSpeech.git
#!ln -s DeepLearningExamples/CUDA-Optimized/FastSpeech/fastspeech fastspeech

from tensorboard.plugins.hparams import api as hp
from tacotron2.hparams import create_hparams
from tacotron2.model import Tacotron2
from tacotron2.stft import STFT
from tacotron2.audio_processing import griffin_lim
from tacotron2.train import load_model
from tacotron2.waveglow.mel2samp import files_to_list, MAX_WAV_VALUE
from tacotron2.waveglow.glow import WaveGlow
from fastspeech.inferencer.denoiser import Denoiser
from fastspeech.text_norm import text_to_sequence
import numpy as np
import torch
import sys
sys.path.append('tacotron2/waveglow/')

def synthesize(text, voice, sigma=0.6, denoiser_strength=0.05, is_fp16=False):

    hparams = create_hparams()
    hparams.sampling_rate = 22050

    if voice == "papaito":
        voice_model = "nvidia_tacotron2_papaito_300"
    elif voice == "constantino":
        voice_model = "tacotron2_Constantino_600"
    elif voice == "orador":
        voice_model = "checkpoint_tacotron2_29000_es"
    
    checkpoint_path = "/home/debian/workspace/models/" + voice_model
        
    model = load_model(hparams)
    model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
    _ = model.cuda().eval().half()

    waveglow_path = '/home/debian/workspace/models/waveglow_256channels_ljs_v2.pt'
    waveglow = torch.load(waveglow_path)['model']
    _ = waveglow.cuda().eval().half()
    denoiser = Denoiser(waveglow)

    #text="¡Cágate lorito!"
    #with open(filelist_path, encoding='utf-8', mode='r') as f:
    #    text = f.read()

    sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
    sequence = torch.autograd.Variable(
        torch.from_numpy(sequence)).cuda().long()

    mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
    #mel = torch.unsqueeze(mel, 0)
    mel = mel_outputs.half() if is_fp16 else mel_outputs
    audio = np.array([])
    with torch.no_grad():
        audio = waveglow.infer(mel, sigma=sigma)
        if denoiser_strength > 0:
             audio = denoiser(audio, denoiser_strength)
        audio = audio * MAX_WAV_VALUE
        audio = audio.squeeze()
        audio = audio.cpu().numpy()
        audio = audio.astype('int16')
    
    return audio, hparams.sampling_rate

## Start chat

In [ ]:
import sounddevice as sd
from sty import fg, bg, ef, rs

print("¡Dadbot está listo para cascar! Escribe tus mensajes o dile 'quieto parao'")
while True:
    a = input()
    
    if a == 'quieto parao':
        break
    responses = await agent.handle_text(a)
    for response in responses:
        to_synth = response["text"]
        #to_synth = "Esto es una prueba para ver si funciona"
        print(fg.blue + to_synth + fg.rs)
        response_file = open('response.txt','w') 
        response_file.write(to_synth)
        voice, sr = synthesize(to_synth, "orador")
        sd.play(voice, sr)
        response_file.close()        

¡Dadbot está listo para cascar! Escribe tus mensajes o dile 'quieto parao'
hola
¿Qué tal?
bien
